<a href="https://colab.research.google.com/github/Chiranjit680/Saahaita/blob/main/SaahaitaModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from collections import Counter
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CHIRANJIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\CHIRANJIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
df=pd.read_csv('tweets_tokenized.csv')

In [ ]:
from nltk.tokenize import word_tokenize
import torch
import pandas as pd

class Vocabulary:
    def __init__(self, df, min_freq=2, tokenizer=word_tokenize):
        self.min_freq = min_freq
        self.tokenizer = tokenizer
        self.df = df
        self.word2index = {'<unk>': 0}

    def build_vocabulary(self):
        # Apply tokenizer to every text row after ensuring it is a string
        self.df["tokens"] = self.df["text"].astype(str).apply(self.tokenizer)

        word_freq = {}

        for tokens in self.df["tokens"]:
            for word in tokens:
                word_freq[word] = word_freq.get(word, 0) + 1

        index = 1  # starting index after <unk>
        for word, freq in word_freq.items():
            if freq >= self.min_freq:
                self.word2index[word] = index
                index += 1

        return self.word2index

    def vectorize_dataframe(self):
        vocab = self.build_vocabulary()

        def vectorize(tokens):
            return [vocab.get(token, 0) for token in tokens]  # 0 = <unk>

        self.df["numerical_text"] = self.df["tokens"].apply(vectorize)

        return self.df
    def zero_padding(self):
      max_len=max(len(x) for x in self.df['numerical_text'])
      self.df['numerical_text']=self.df['numerical_text'].apply(lambda x:x+[0]*(max_len-len(x)))
      return self.df
    def get_vocab_size(self):
      return len(self.word2index)


In [ ]:
vocab_builder=Vocabulary(df)

In [ ]:
vocab=vocab_builder.build_vocabulary()
vocab_builder.vectorize_dataframe()
vocab_builder.zero_padding()

,text,disaster_category,label,tokens,numerical_text
0,communal violence bhainsa telangana stone pelt...,Fires,3,"[communal, violence, bhainsa, telangana, stone...","[1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 0, 0, 0..."
1,telangana section imposed bhainsa january clas...,Fires,3,"[telangana, section, imposed, bhainsa, january...","[4, 12, 13, 3, 14, 15, 16, 17, 18, 14, 19, 0, ..."
2,arsonist set car ablaze dealership,Fires,3,"[arsonist, set, car, ablaze, dealership]","[20, 10, 21, 11, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,arsonist set car ablaze dealership,Fires,3,"[arsonist, set, car, ablaze, dealership]","[20, 10, 21, 11, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,lord jesus love brings freedom pardon fill hol...,Fires,3,"[lord, jesus, love, brings, freedom, pardon, f...","[23, 24, 25, 26, 27, 0, 28, 29, 30, 10, 31, 11..."
...,...,...,...,...,...
9546,medium warned u well advance wrecked whole nig...,Explosions,2,"[medium, warned, u, well, advance, wrecked, wh...","[40, 4410, 238, 538, 2548, 5771, 110, 55, 3870..."
9547,feel directly attacked consider moonbin amp ji...,Explosions,2,"[feel, directly, attacked, consider, moonbin, ...","[497, 3303, 1937, 774, 7872, 105, 7887, 1272, ..."
9548,feel directly attacked consider moonbin amp ji...,Explosions,2,"[feel, directly, attacked, consider, moonbin, ...","[497, 3303, 1937, 774, 7872, 105, 7887, 1272, ..."
9549,ok remember outcast nd dora au au wrecked nerv...,Explosions,2,"[ok, remember, outcast, nd, dora, au, au, wrec...","[730, 878, 1546, 3646, 0, 1190, 1190, 5771, 70..."


In [ ]:
df

,text,disaster_category,label,tokens,numerical_text
0,communal violence bhainsa telangana stone pelt...,Fires,3,"[communal, violence, bhainsa, telangana, stone...","[1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 0, 0, 0..."
1,telangana section imposed bhainsa january clas...,Fires,3,"[telangana, section, imposed, bhainsa, january...","[4, 12, 13, 3, 14, 15, 16, 17, 18, 14, 19, 0, ..."
2,arsonist set car ablaze dealership,Fires,3,"[arsonist, set, car, ablaze, dealership]","[20, 10, 21, 11, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,arsonist set car ablaze dealership,Fires,3,"[arsonist, set, car, ablaze, dealership]","[20, 10, 21, 11, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,lord jesus love brings freedom pardon fill hol...,Fires,3,"[lord, jesus, love, brings, freedom, pardon, f...","[23, 24, 25, 26, 27, 0, 28, 29, 30, 10, 31, 11..."
...,...,...,...,...,...
9546,medium warned u well advance wrecked whole nig...,Explosions,2,"[medium, warned, u, well, advance, wrecked, wh...","[40, 4410, 238, 538, 2548, 5771, 110, 55, 3870..."
9547,feel directly attacked consider moonbin amp ji...,Explosions,2,"[feel, directly, attacked, consider, moonbin, ...","[497, 3303, 1937, 774, 7872, 105, 7887, 1272, ..."
9548,feel directly attacked consider moonbin amp ji...,Explosions,2,"[feel, directly, attacked, consider, moonbin, ...","[497, 3303, 1937, 774, 7872, 105, 7887, 1272, ..."
9549,ok remember outcast nd dora au au wrecked nerv...,Explosions,2,"[ok, remember, outcast, nd, dora, au, au, wrec...","[730, 878, 1546, 3646, 0, 1190, 1190, 5771, 70..."


In [ ]:
X=df['numerical_text']
y=df['label']

In [ ]:
X=torch.tensor(X.tolist())
y=torch.tensor(y.tolist())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
type(X_train)

torch.Tensor

In [ ]:
vocab_size=vocab_builder.get_vocab_size()

In [ ]:
vocab_size

7888

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.data[index], self.labels[index]

In [ ]:
dataset=CustomDataset(X_train, y_train)

In [ ]:
dataloader=DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size=26, embedding_dim=256, hidden_dim=512, output_dim=10, n_layers=4, bidirectional=False, dropout=0.15, pad_idx=0):
        super().__init__()
        self.bidirectional =bidirectional  # Store bidirectional as instance variable
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout if n_layers > 1 else 0)
        self.attention=nn.MultiheadAttention(hidden_dim, 4, dropout=dropout)
        self.fc= nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.BatchNorm1d(hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, output_dim)
        )
        self.dropout = nn.Dropout(dropout)


    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_out, (hidden, cell) = self.rnn(embedded)

        if self.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]

        # Fix attention mechanism - all inputs should have same dimensions
        attn_input = lstm_out.transpose(0, 1)  # [seq_len, batch, hidden_dim]
        attn_out, _ = self.attention(attn_input, attn_input, attn_input)
        attn_out = attn_out.transpose(0, 1)  # [batch, seq_len, hidden_dim]

        # Global max pooling over sequence dimension
        out = torch.max(attn_out, dim=1)[0]  # [batch, hidden_dim]
        output = self.fc(out)

        return output

In [ ]:
model=LSTMModel(vocab_size)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

LSTMModel(
  (embedding): Embedding(7888, 256, padding_idx=0)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.15)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (fc): Sequential(
    (0): Dropout(p=0.15, inplace=False)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Dropout(p=0.075, inplace=False)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=10, bias=True)
  )
  (dropout): Dropout(p=0.15, inplace=False)
)

In [ ]:
for label in df['label'].unique():
  print(label,":",(df['label']==label).sum())

3 : 1141
0 : 1144
5 : 1536
4 : 1671
2 : 1127
8 : 1453
6 : 312
9 : 70
1 : 883
7 : 214


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import numpy as np

def setup_training(model, learning_rate=0.01):
    model.to(device)



    classes = torch.unique(y_train).numpy()
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train.numpy())
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True
    )

    return optimizer, criterion, scheduler

In [ ]:
def train(model, dataloader, optimizer, criterion, scheduler, epochs=5):
    for epoch in range(epochs):
        model.train()  # Ensure training mode
        total_loss = 0
        correct = 0
        total = 0

        for data, labels in dataloader:
            data = data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)

        avg_loss = total_loss / len(dataloader)
        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}")

        scheduler.step(avg_loss)


In [ ]:
train(model, dataloader, *setup_training(model), epochs=100)

c:\Users\CHIRANJIT\anaconda3\envs\myenv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
def evaluate_model(model, test_loader, criterion=None, device='cpu'):
    """
    Evaluate the LSTM model on test data

    Args:
        model: Trained LSTM model
        test_loader: DataLoader for test data
        criterion: Loss function (optional)
        device: Device to run evaluation on

    Returns:
        Dictionary containing evaluation metrics
    """
    model.eval()
    model.to(device)

    all_predictions = []
    all_true_labels = []
    total_loss = 0.0

    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)

            # Forward pass
            outputs = model(data)

            # Calculate loss if criterion provided
            if criterion is not None:
                loss = criterion(outputs, targets)
                total_loss += loss.item()

            # Get predictions
            predictions = torch.argmax(outputs, dim=1)

            # Store results
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    all_predictions = np.array(all_predictions)
    all_true_labels = np.array(all_true_labels)

    # Calculate metrics
    accuracy = accuracy_score(all_true_labels, all_predictions)
    precision, recall, f1, support = precision_recall_fscore_support(
        all_true_labels, all_predictions, average='weighted', zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(all_true_labels, all_predictions)

    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'average_loss': total_loss / len(test_loader) if criterion is not None else None
    }

    return results

In [ ]:
testloader = DataLoader(CustomDataset(X_test, y_test), batch_size=32, shuffle=False)
criterion=nn.CrossEntropyLoss()
results = evaluate_model(model, testloader, criterion=criterion , device=device)

NameError: name 'X_test_tensor' is not defined

In [ ]:
print("Evaluation Results:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1_score']:.4f}")

